In [1]:
%reload_ext autoreload
%autoreload 2

In [57]:
import pandas as pd
import numpy as np

In [30]:
clusters_stat1 = pd.DataFrame([
    {"component": "s-carts", "rep_metric": "s-carts_latency", "sub_metrics": ["s-carts_errors", "s-carts_throughput"]},
    {"component": "c-user", "rep_metric": "c-user_cpu_user", "sub_metrics": ["c-user_cpu_sys", "c-user_cpu_wait"]},
    {"component": "c-user", "rep_metric": "c-user_mem", "sub_metrics": []},
])
clusters_stat1["dataset_id"] = "xxx"
clusters_stat1["chaos_type"] = "pod-cpu-hog"
clusters_stat1["chaos_comp"] = "user"
clusters_stat1["chaos_case_num"] = "0"
clusters_stat2 = pd.DataFrame([
    {"component": "s-carts", "rep_metric": "s-carts_latency", "sub_metrics": ["s-carts_errors", "s-carts_throughput", "s-carts_lat99"]},
    {"component": "c-user", "rep_metric": "c-user_cpu_user", "sub_metrics": ["c-user_cpu_sys", "c-user_cpu_wait"]},
    {"component": "c-user", "rep_metric": "c-user_mem", "sub_metrics": []},
])
clusters_stat2["dataset_id"] = "xxx"
clusters_stat2["chaos_type"] = "pod-cpu-hog"
clusters_stat2["chaos_comp"] = "user"
clusters_stat2["chaos_case_num"] = "1"

In [31]:
clusters_stats = pd.DataFrame(pd.concat([clusters_stat1, clusters_stat2], axis=0))
clusters_stats

,component,rep_metric,sub_metrics,dataset_id,chaos_type,chaos_comp,chaos_case_num
0,s-carts,s-carts_latency,"[s-carts_errors, s-carts_throughput]",xxx,pod-cpu-hog,user,0
1,c-user,c-user_cpu_user,"[c-user_cpu_sys, c-user_cpu_wait]",xxx,pod-cpu-hog,user,0
2,c-user,c-user_mem,[],xxx,pod-cpu-hog,user,0
0,s-carts,s-carts_latency,"[s-carts_errors, s-carts_throughput, s-carts_l...",xxx,pod-cpu-hog,user,1
1,c-user,c-user_cpu_user,"[c-user_cpu_sys, c-user_cpu_wait]",xxx,pod-cpu-hog,user,1
2,c-user,c-user_mem,[],xxx,pod-cpu-hog,user,1


In [32]:
clusters_stats.set_index(["dataset_id", "chaos_type", "chaos_comp", "chaos_case_num"], inplace=True)
clusters_stats

component       rep_metric  \
dataset_id chaos_type  chaos_comp chaos_case_num                              
xxx        pod-cpu-hog user       0                s-carts  s-carts_latency   
                                  0                 c-user  c-user_cpu_user   
                                  0                 c-user       c-user_mem   
                                  1                s-carts  s-carts_latency   
                                  1                 c-user  c-user_cpu_user   
                                  1                 c-user       c-user_mem   

                                                                                        sub_metrics  
dataset_id chaos_type  chaos_comp chaos_case_num                                                     
xxx        pod-cpu-hog user       0                            [s-carts_errors, s-carts_throughput]  
                                  0                               [c-user_cpu_sys, c-user_cpu_wait]  
                                  0                                                              []  
                                  1               [s-carts_errors, s-carts_throughput, s-carts_l...  
                                  1                               [c-user_cpu_sys, c-user_cpu_wait]  
                                  1                                                              []

In [64]:
# num_clusters
num_clusters = clusters_stats.groupby(["dataset_id", "chaos_type", "chaos_comp", "chaos_case_num", "component"]).size()
num_clusters

dataset_id  chaos_type   chaos_comp  chaos_case_num  component
xxx         pod-cpu-hog  user        0               c-user       2
                                                     s-carts      1
                                     1               c-user       2
                                                     s-carts      1
dtype: int64

In [65]:
# cluster size
cluster_size = clusters_stats.groupby(["dataset_id", "chaos_type", "chaos_comp", "chaos_case_num", "component", "rep_metric"])["sub_metrics"].apply(lambda x: np.array(x[0]).flatten().size + 1)
cluster_size

dataset_id  chaos_type   chaos_comp  chaos_case_num  component  rep_metric     
xxx         pod-cpu-hog  user        0               c-user     c-user_cpu_user    3
                                                                c-user_mem         1
                                                     s-carts    s-carts_latency    3
                                     1               c-user     c-user_cpu_user    3
                                                                c-user_mem         1
                                                     s-carts    s-carts_latency    4
Name: sub_metrics, dtype: int64

In [69]:
num_clusters.agg(["mean", "std", "min", "max"]).to_dict()

{'mean': 1.5, 'std': 0.5773502691896257, 'min': 1.0, 'max': 2.0}

In [68]:
cluster_size.agg(["mean", "std", "min", "max"])

mean    2.500000
std     1.224745
min     1.000000
max     4.000000
Name: sub_metrics, dtype: float64